In [28]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join("..", "..")))
from nlp.preprocessing import jamo_spliter
import json

In [ ]:
r = open("../../datasets/extractive_summary/train.jsonl", "r", encoding="utf8")
w = open("../../datasets/extractive_summary/train_hg.json", "a", encoding="utf8")
c = open("../../datasets/extractive_summary/corpus.txt", "a", encoding="utf8")
try:
    count = 0
    while True:
        line = r.readline()
        if not line: break
        count += 1
        line = json.loads(line)
        for i, s in enumerate(line["article_original"]):
            line["article_original"][i] = hg_encoder.encode(s)
            c.write(line["article_original"][i] + "\n")
        line["abstractive"] = hg_encoder.encode(line["abstractive"])
        c.write(line["abstractive"] + "\n")
        w.write(json.dumps(line, ensure_ascii=False))
        if count % 10000 == 0:
            print(f"█{count}")
        elif count % 100 == 0:
            print("█", end="")
    print(f"data size: {count}")
except Exception as e:
    raise e
finally:
    r.close()
    w.close()
    c.close()

In [ ]:
with open("../../datasets/extractive_summary/corpus.txt", "r", encoding="utf8") as c:
    dataset = c.readlines()

len(dataset)

### Make WordPiece tokenizer

https://velog.io/@nawnoes/Huggingface-tokenizers%EB%A5%BC-%EC%82%AC%EC%9A%A9%ED%95%9C-Wordpiece-Tokenizer-%EB%A7%8C%EB%93%A4%EA%B8%B0

In [18]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece

bert_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

In [19]:
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase, NFD, StripAccents

bert_tokenizer.normalizer = normalizers.Sequence([NFD(), Lowercase(), StripAccents()])

In [20]:
from tokenizers.pre_tokenizers import Whitespace

bert_tokenizer.pre_tokenizer = Whitespace()

In [21]:
from tokenizers.processors import TemplateProcessing

bert_tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", 1),
        ("[SEP]", 2),
    ],
)

In [22]:
from tokenizers.trainers import WordPieceTrainer

trainer = WordPieceTrainer(
    vocab_size=30522, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)
files = ["../../datasets/extractive_summary/corpus.txt"]
bert_tokenizer.train(files, trainer)

bert_tokenizer.save("../../datasets/extractive_summary/wpm.json")

In [27]:
output = bert_tokenizer.encode(hg_encoder.encode("이거 잘 나오나요?"))
print(output.tokens)
# [1, 27253, 16, 93, 11, 5097, 5, 7961, 5112, 6218, 0, 35, 2]

bert_tokenizer.decode(output.ids)
# "Hello , y ' all ! How are you ?"

['[CLS]', 'ㅇㅣ', '-', 'ㄱㅓ', '-', 'ㅈㅏㄹ', 'ㄴㅏ', '-', 'ㅇㅗ', '-', 'ㄴㅏ', '-', 'ㅇㅛ', '--?-', '[SEP]']


'ㅇㅣ - ㄱㅓ - ㅈㅏㄹ ㄴㅏ - ㅇㅗ - ㄴㅏ - ㅇㅛ --?-'